In [ ]:
from typing import List, Optional, Callable, Tuple
from torch.utils.data import DataLoader

# Utilities

In [3]:
from functools import wraps
from time import time

def timing(f: Callable, *args, **kwargs) -> Callable:
    @wraps(f)
    def wrap(*args, **kwargs):
        ts = time()
        result = f(*args, **kwargs)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kwargs, te-ts))
        return result
    return wrap

# Dataset for the Parquet files

In [1]:
from parquet_dataset.parquet_dataset import ParquetDataset
%load_ext autoreload
%autoreload 2

# Instantiate the dataloader.

In [4]:
def make_train_parquet_path(i: int) -> str:
    return f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
# Setup the file indices to use.
K_MAX_TRAIN_FILES: int = 10
K_TRAIN_FILES: List[str] = [make_train_parquet_path(i) for i in range(K_MAX_TRAIN_FILES)]
K_TEST_FILES: List[str] = ["/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"]

train_dataset = ParquetDataset(file_paths=K_TRAIN_FILES, logging=True)
# test_dataset = ParquetDataset(file_paths=K_TEST_FILES, logging=True)

Loaded files with rows:
	1944210 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet
	2804247 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
	3036873 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
	4016784 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
	5022952 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
	5348200 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
	6203912 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
	6335560 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=7/part-0.parquet
	6140024 : /kaggle/input/jane-street-real-time-market-da

## (Optional) Run timing analysis to check batched approach vs. naive single-row loading

In [15]:
@timing
def iterate_samples_batched_row_group(step_size: int):
    for i in range(0, len(train_dataset), step_size):
        train_dataset[i]

iterate_samples_batched_row_group(step_size=1000)
iterate_samples_batched_row_group(step_size=100)

func:'iterate_samples_batched_row_group' args:[(), {'step_size': 1000}] took: 23.3475 sec
func:'iterate_samples_batched_row_group' args:[(), {'step_size': 100}] took: 37.6389 sec


# Create a custom sample for randomly ordering the parquet files and rows within the parquet files

In [7]:
from parquet_sampler.parquet_sampler import ParquetBatchedSampler
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## (Optional) Verify that the sampler produces samples in random parition order

In [11]:
sampler = ParquetBatchedSampler(data_source=train_dataset)
partition_ordering: List[Tuple[int,int]] = []
num_sampled_per_partition: List[int] = []
curr_partition: Optional[Tuple[int, int]] = None

for i, idx in enumerate(sampler):

    if (i % 1000000 == 0):
        print(f"{i} of {len(sampler)}\r")

    new_idx = train_dataset.calculate_index_from_cumulative_counts(idx, train_dataset.cum_total_counts_np)
    if curr_partition != new_idx:
        # update the partition index assuming that once we see a file index and then leave it
        # we'll never see it again. Throw an error if we do see it again.
        if new_idx in partition_ordering:
            raise ValueError(f"File index {new_idx} seen out of order! Already seen ordering: {partition_ordering}")
        print(f"found new file {new_idx}")
        curr_partition = new_idx
        partition_ordering.append(curr_partition)
        if len(num_sampled_per_partition) > 0:
            num_sampled_per_partition.append(i - num_sampled_per_partition[-1])
        else:
            num_sampled_per_partition.append(i)

print(f"Total elements: {len(sampler)}. Saw {len(partition_ordering)}, partition ordering: {partition_ordering}")

0 of 47127338
found new file 115
found new file 50
found new file 25
found new file 125
1000000 of 47127338
found new file 75
found new file 63
found new file 2
found new file 51
2000000 of 47127338
found new file 154
found new file 70
found new file 22
found new file 155
3000000 of 47127338
found new file 158
found new file 9
found new file 74
4000000 of 47127338
found new file 37
found new file 5
found new file 111
found new file 35
5000000 of 47127338
found new file 62
found new file 20
found new file 110
found new file 55
6000000 of 47127338
found new file 86
found new file 171
found new file 64
7000000 of 47127338
found new file 141
found new file 48
found new file 80
found new file 131
8000000 of 47127338
found new file 61
found new file 173
found new file 0
found new file 135
9000000 of 47127338
found new file 26
found new file 18
found new file 68
found new file 77
10000000 of 47127338
found new file 124
found new file 98
found new file 157
11000000 of 47127338
found new file 3

## (Optional) Benchmarking how long it takes to serve samples from the data loader

In [ ]:

# Apparently there's an issue with 
@timing
def iterate_dataloader(num_workers=0, batch_size=1, prefetch_factor=1):
    dl = DataLoader(
        train_dataset, 
        batch_size=batch_size,
        num_workers=num_workers,
        # prefetch_factor=prefetch_factor,
        sampler=ParquetBatchedSampler(data_source=train_dataset)
    )
    print("Created dataloader.")
    for i, batch in enumerate(dl):
        if i % 1000 == 0:
            print(i, len(dl))

iterate_dataloader()

# Prepare data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
# Separate features and responders
features = sample_df.filter(regex='^feature_')
responders = sample_df.filter(regex='^responder_')
# Convert to numpy arrays for TensorFlow
X = features.values  # Features for input
y = responders.values  # Responders for output
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

# Build the Autoencoder Model

In [ ]:
# Define the number of input and output nodes
input_dim = X.shape[1]  # Number of features (79)
output_dim = y.shape[1]  # Number of responders (9)
# Define the model
model = models.Sequential([
    layers.Input(shape=(input_dim,)),  # Input layer
    layers.Dense(64, activation='relu'),  # Encoder
    layers.Dense(32, activation='relu'),  # Bottleneck layer (compression)
    layers.Dense(64, activation='relu'),  # Decoder
    layers.Dense(output_dim, activation='linear')  # Output layer for responders
])
model.compile(optimizer='adam', loss='mse')

# Train Autoencoder Model

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=10,            # Number of epochs to wait for improvement
    min_delta=0.001,       # Minimum change to qualify as an improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
model.save("/kaggle/working/model.keras")

# Submission

See [Jane Street RMF Demo Submission](https://www.kaggle.com/code/ryanholbrook/jane-street-rmf-demo-submission) for details

In [ ]:
import os
import polars as pl
import kaggle_evaluation.jane_street_inference_server

In [ ]:
import polars as pl
import numpy as np
# Assuming `model` is your trained model
# Assuming features required by the model are named 'feature_00', 'feature_01', etc.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags
    # Extract the features for the model input
    feature_columns = [col for col in test.columns if col.startswith("feature_")]
    features = test.select(feature_columns).to_numpy()  # Convert to numpy array for model input
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    # Generate predictions using the model
    model_predictions = model.predict(features)
    responder_6_predictions = model_predictions[:, 6]  # Assuming responder_6 is at index 6
    # Create a new Polars DataFrame with row_id and responder_6 predictions
    predictions = test.select("row_id").with_columns(
        pl.Series("responder_6", responder_6_predictions)
    )
    # Ensure the output format and length requirements
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    
    assert len(predictions) == len(test)
    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )